#Téléchargement des bibliothèques

In [1]:
# Installation des paquets nécessaires via pip. Ces commandes installent les bibliothèques LangChain,
# ChromaDB pour la gestion des embeddings, Sentence-Transformers pour les embeddings sémantiques,
# et les bibliothèques Transformers, Torch et Accelerate pour le traitement du langage naturel et l'apprentissage automatique.
!pip install langchain
!pip install chromadb
!pip install sentence-transformers
!pip install transformers torch accelerate
!pip install accelerate
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#Importer les bibliothèques

In [2]:
# Importation des bibliothèques nécessaires pour le traitement des données et le NLP.
import pandas as pd
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer
from transformers import pipeline
import torch
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory

#Pre-processing

telecharger la dataframe sur les film

In [ ]:
# Téléchargement et affichage d'un exemple de DataFrame contenant des informations sur des films.
df = pd.read_csv("/dataframe1.csv")

print("exemple df", df)

supprimer la colonnes qui contient des liens

In [ ]:
# Suppression des colonnes non nécessaires, comme les liens wiki.
df = df.drop(['Wiki Page'], axis=1)

Remplacer les valeurs manquantes par des valeurs par défaut ou spécifiques

In [ ]:
# Remplacement des valeurs manquantes par des valeurs par défaut ou spécifiques dans le DataFrame.
df = df.fillna({'Release Year': 0, 'Title': 'Unknown', 'Origin/Ethnicity': 'Unknown', 'Director': 'Unknown', 'Cast': 'Unknown', 'Genre': 'Unknown', 'Plot': 'Unknown'})

Conversion de la colonne 'Release Year' en type numérique

In [ ]:
# Conversion de la colonne 'Release Year' en type numérique, gestion des erreurs si nécessaire.
df['Release Year'] = pd.to_numeric(df['Release Year'], errors='coerce')

Conversion des autres colonnes en type de données approprié (par exemple, chaîne de caractères)

In [ ]:
# Conversion des autres colonnes en chaînes de caractères pour une manipulation plus aisée.
df['Title'] = df['Title'].astype(str)
df['Origin/Ethnicity'] = df['Origin/Ethnicity'].astype(str)
df['Director'] = df['Director'].astype(str)
df['Cast'] = df['Cast'].astype(str)
df['Genre'] = df['Genre'].astype(str)
df['Plot'] = df['Plot'].astype(str)

Supprimer les doublons

In [ ]:
# Suppression des entrées en double dans le DataFrame.
df = df.drop_duplicates()

rendre tous les mots en miniscules

In [ ]:
# Conversion de toutes les chaînes de caractères en minuscules pour uniformiser les données.
df['Title'] = df['Title'].str.lower()
df['Director'] = df['Director'].str.lower()
df['Cast'] = df['Cast'].str.lower()
df['Genre'] = df['Genre'].str.lower()
df['Plot'] = df['Plot'].str.lower()

Telecharger la nouvelle base de donner "modified_dataframe.csv"

In [ ]:
# Sauvegarde du DataFrame modifié dans un nouveau fichier CSV.
df.to_csv('modified_dataframe.csv', index=False)

Importer la nouvelle dataFrame

In [7]:
# Rechargement des données depuis le nouveau fichier CSV et affichage d'un exemple.
df = pd.read_csv("/content/modified_dataframe.csv")
print("exemple df", df)

exemple df        Release Year                             Title Origin/Ethnicity  \
0              1901            kansas saloon smashers         American   
1              1901     love by the light of the moon         American   
2              1901           the martyred presidents         American   
3              1901  terrible teddy, the grizzly king         American   
4              1902            jack and the beanstalk         American   
...             ...                               ...              ...   
34881          2014                 the water diviner          Turkish   
34882          2017               çalgı çengi i̇kimiz          Turkish   
34883          2017                      olanlar oldu          Turkish   
34884          2017                  non-transferable          Turkish   
34885          2017               i̇stanbul kırmızısı          Turkish   

                                 Director  \
0                                 unknown   
1         

#Quantisation du modèle réduire l'utilisation de la RAM

In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [4]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#Connexion au Hagging Face pour télécharger le modèle Llama2 7B Chat

In [ ]:
!huggingface-cli login

In [6]:
model_name = "meta-llama/Llama-2-7b-chat-hf" # meta-llama/Llama-2-7b-hf
access_token = "haggingface_token"

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    use_auth_token=access_token
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=access_token)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

#Création du pipeline

In [8]:
from transformers import pipeline

llama_pipeline = pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    #temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1,  # without this output begins repeating
    torch_dtype=torch.float16,
    device_map="auto",
)

In [9]:
# Création d'un pipeline spécifique à HuggingFace pour l'exécution des tâches.
llm = HuggingFacePipeline(pipeline=llama_pipeline)

## Traitement et segmentation du document

In [10]:
# Chargement du document.
loader = CSVLoader("/content/modified_dataframe.csv", encoding="utf-8")
documents = loader.load()

# Segmentation du texte en morceaux de taille définie.
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# Création d'une fonction d'encodage des textes en vecteurs avec un modèle de transformer.
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

## Stockage des embeddings dans une base de données Chroma

In [11]:
# Création et remplissage d'une base de données Chroma db avec les documents et leurs embeddings.
db = Chroma.from_documents(docs, embedding_function, persist_directory="chroma_db")

## Création du template



In [16]:
# Définition d'un template de prompt pour guider les interactions avec le modèle de langage.
template = """
Have a conversation with a human. If you don't find the answer in the database, just say that you don't know.
Don't try to make up an answer. Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question:
------
<ctx>
{context}
</ctx>
------
<hs>
{history}
</hs>
------
{question}
Answer:
"""
# Création d'un objet PromptTemplate pour structurer les entrées fournies au modèle.
prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

# Création de la mémoire de conversation ConversationBuffersMemory

In [17]:
# Configuration de la mémoire de conversation pour stocker l'historique des interactions.
memory = ConversationBufferMemory(
    memory_key="history",  # Clé pour l'élément de mémoire à utiliser
    input_key="question"  # Clé pour l'entrée de la question
)

# Création d'une chaîne de récupération et de réponse aux questions en utilisant le modèle et la base de données.
retrieval_chain = RetrievalQA.from_chain_type(llm,
                                              chain_type='stuff',  # Type de chaîne pour le traitement
                                              retriever=db.as_retriever(),  # Utilisation de la base de données comme récupérateur
                                              chain_type_kwargs={
                                                  "prompt": prompt,  # Utilisation du template de prompt
                                                  "memory": memory  # Utilisation de la mémoire de conversation
                                              })

#Test

In [18]:
# Exécution d'une requête de test pour interagir avec le modèle.
query = "hello, my name is OUARDIRHI!"
retrieval_chain.run(query)

"Hello OUARDIRHI! I'm happy to help you with your questions. However, I must inform you that the movies you have listed do not exist in reality. They appear to be fictional movies with absurd plots and characters. As a responsible AI language model, I cannot provide answers to non-existent movies. Please feel free to ask me any other questions, and I'll do my best to assist you."

In [19]:
# Autre requête de test pour obtenir des informations sur un film
query = "give me the origin/Ethnicity of the film jack and the beanstalk "
retrieval_chain.run(query)

'The origin of the film Jack and the Beanstalk is American.'

In [20]:
query = "give me the Director of the film jack and the beanstalk "
retrieval_chain.run(query)

'The director of the film Jack and the Beanstalk is George S. Fleming, Edwin S. Porter.\n\nPlease let me know if you need further assistance.'

In [21]:
query = "give me the Plot of the film jack and the beanstalk "
retrieval_chain.run(query)

"The plot of the film Jack and the Beanstalk is as follows:\n\nJack trades his only cow for magic beans, which grow overnight into a beanstalk reaching the clouds. He climbs the beanstalk and finds a giant's castle, where he encounters a fairy, a princess, and a golden egg-laying hen. The giant chases Jack down the beanstalk, but Jack cuts it down and escapes. The fairy reveals that Jack may return home as a prince.\n\nPlease let me know if you need further assistance."

In [28]:
# Exécution d'une requête de test pour avoir une recommendation sur un film.
query = "give me a film taklking about war "
retrieval_chain.run(query)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'The film "Last War!" The Last War" is a fictional movie with a plot that involves a war between two countries. The film takes place in a dystopian future where the world has been ravaged by war, and a small group of survivors must fight for their lives against an oppressive government. The film explores themes of resistance, rebellion, and the human spirit in the face of unimaginable adversity. It is a powerful and thought-provoking work that challenges the viewer to consider the consequences of war and the importance of standing up for what is right.'

In [29]:
# Exécution d'une requête de test pour tester si le modèle hallucine.
query = "do you know this film JHDKJCDSKJCNKZHIIH"
retrieval_chain.run(query)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'I apologize, but I\'m not familiar with a film called "JHDKJCDSKJCNKZHIIH." As a responsible AI language model, I cannot provide information on non-existent films or TV shows. Can you please provide more context or clarify the title?'

In [30]:
# Exécution d'une requête de test pour tester la mémoire du modèle.
query = "What is my name again?"
retrieval_chain.run(query)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


"Hello OUARDIRHI! I'm happy to help you with your questions. However, I must inform you that the movies you have listed do not exist in reality. They appear to be fictional movies with absurd plots and characters. As a responsible AI language model, I cannot provide answers to non-existent movies. Please feel free to ask me any other questions, and I'll do my best to assist you.\n\nYour name is OUARDIRHI."

In [31]:
# Exécution d'une requête de test pour tester la mémoire du modèle.
query = "what was the last film you recommended?"
retrieval_chain.run(query)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'The last film I recommended was "Last War!"'

In [ ]:
# Exécution d'une requête de test pour tester la mémoire du modèle.
query = "what was the last film you recommended?"
retrieval_chain.run(query)